<a href="https://colab.research.google.com/github/SofiaKung/ai-agent-course/blob/main/Day_1_Introduction_to_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.1: Install dependencies¶


Install the latest pip installer and google-adk library

In [1]:
!python -m pip install --upgrade pip
!pip install --upgrade google-adk

## 1.2: Configure your Gemini API Key¶



In [3]:
import os
from google.colab import userdata
userdata.get('GOOGLE_API_KEY')

try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


## 1.3: Import ADK components¶


In [4]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("✅ ADK components imported successfully.")


✅ ADK components imported successfully.


**What are these components for** :

* Agent (from google.adk.agents): Represents an AI “worker” that can take instructions, call tools, and
manage interactions with models.

* Gemini (from google.adk.models.google_llm): A wrapper/client for Google’s Gemini LLM, used to send prompts and get model responses.

* InMemoryRunner (from google.adk.runners): Runs agents and workflows entirely in memory, useful for quick local execution and testing.

* google_search (from google.adk.tools): A tool that lets your agent call Google Search as an external capability to fetch web information.

* types (from google.genai): Provides shared data structures (like request/response schemas, configs, etc.) used when interacting with Google’s generative AI APIs.

## 1.4: Helper functions
We'll define some helper functions. If you are running this outside the Kaggle environment, you don't need to do this.

In [5]:
# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("✅ Helper functions defined.")

✅ Helper functions defined.


## 1.5: Configure Retry Options
When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability. Retry options automatically handle these failures by retrying the request with exponential backoff.

In [6]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

## 2.1 What is an AI Agent?¶
An AI Agent takes this one step further. An agent can think, take actions, and observe the results of those actions to give you a better answer.

Prompt -> Agent -> Thought -> Action -> Observation -> Final Answer



## 2.2 Define your agent
Now, let's build our agent.

These are the main properties we'll set:

* name and description: A simple name and description to identify our agent.
* model: The specific LLM that will power the agent's reasoning. We'll use "gemini-2.5-flash-lite".
* instruction: The agent's guiding prompt. This tells the agent what its goal is and how to behave.
* tools: A list of tools that the agent can use. To start, we'll give it the google_search tool, which lets it find up-to-date information online.

In [7]:
root_agent = Agent(
    name="helpful_assistant",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="A simple agent that can answer general questions.",
    instruction="You are a helpful assistant. Use Google Search for current info or if unsure.",
    tools=[google_search],
)

print("✅ Root Agent defined.")

✅ Root Agent defined.


## 2.3 Run your agent
Now it's time to bring your agent to life and send it a query. To do this, you need a Runner, which is the central component within ADK that acts as the orchestrator. It manages the conversation, sends our messages to the agent, and handles its responses.

a. Create an InMemoryRunner and tell it to use our root_agent:



In [8]:
runner = InMemoryRunner(agent=root_agent)

print("✅ Runner created.")

✅ Runner created.


b. Now you can call the .run_debug() method to send our prompt and get an answer.

👉 This method abstracts the process of session creation and maintenance and is used in prototyping. We'll explore "what sessions are and how to create them" on Day 3.

In [11]:
response = await runner.run_debug(
    "What is the latest AI trends for today?"
)


 ### Continue session: debug_session_id

User > What is the latest AI trends for today?
helpful_assistant > Here are some of the latest AI trends for today and looking ahead into 2025:

**AI Agents and Autonomous Systems:** A significant shift is occurring with AI agents moving beyond simple chatbots to become more autonomous. These agents can understand, plan, and execute tasks with less direct human prompting, acting more like digital colleagues. This evolution is expected to lead to multi-agent systems where specialized AI agents coordinate to handle complex tasks, with some predictions suggesting that by 2025, 25% of companies using generative AI will launch agentic AI pilots.

**Multimodal AI:** AI is increasingly breaking down barriers between different data types. Multimodal AI systems can simultaneously process and integrate text, images, audio, video, and sensor data, allowing for a more comprehensive understanding of context. This trend is driven by the understanding that th



## 2.4 How does it work?
The agent performed a Google Search to get the latest information about ADK, and it knew to use this tool because:

The agent inspects and is aware of which tools it has available to use.
The agent's instructions specify the use of the search tool to get current information or if it is unsure of an answer.
The best way to see the full, detailed trace of the agent's thoughts and actions is in the ADK web UI, which we'll set up later in this notebook.

Other resources:
* [ADK doc]( https://google.github.io/adk-docs/api-reference/python/search.html?q=run&check_keywords=yes&area=default#)
* [Day 1 notebook](https://www.kaggle.com/code/sofiakung/day-1a-from-prompt-to-action-632f6a/edit)
* [Day 1 live stream]( https://www.youtube.com/live/ZaUcqznlhv8)
* [ADK Documentation](https://https://google.github.io/adk-docs/)
* [ADK Agents Overview](https://google.github.io/adk-docs/agents/)
* [ADK Tools Overview]( https://google.github.io/adk-docs/tools/)